<a href="https://colab.research.google.com/github/Hsayed1/Medical-Mnist-Image-Classifier/blob/main/Medical_Image_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
%matplotlib inline
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from google.colab import drive
import shutil
from sklearn.metrics import classification_report
import random
"""This code loads, preprocesses, and trains a cnn on the medical mnist dataset. 4 experiments were tried in this file and have been reported. Two cnn models were used"""

In [ ]:
drive.mount('/content/drive')

In [ ]:
##importing data from drive
zip_content = zipfile.ZipFile('/content/drive/MyDrive/medicalimages.zip', 'r')
zip_content.extractall('data')
zip_content.close()

In [ ]:
data = '/content/data/medicalimages'
batch_size = 32
img_height = 150
img_width = 150

In [ ]:
#randomly sample 5k images from each class because dataset was too large.

# Define the class names
class_names = ['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']

# Define the target number of images per class
images_per_class = 5000

# Create a new directory for the reduced dataset
reduced_data_path = '/content/data/reduced_medicalimages'
os.makedirs(reduced_data_path, exist_ok=True)

# Loop through each class
for class_name in class_names:
    class_path = os.path.join(data, class_name)
    target_class_path = os.path.join(reduced_data_path, class_name)

    os.makedirs(target_class_path, exist_ok=True)

    # List all files in the class directory and shuffle the list to randomize it
    class_files = os.listdir(class_path)
    random.shuffle(class_files)

    # Copy the desired number of randomly selected images (5,000) from the class directory to the reduced dataset directory
    for i in range(images_per_class):
        file_name = class_files[i]
        source_file_path = os.path.join(class_path, file_name)
        target_file_path = os.path.join(target_class_path, file_name)
        shutil.copyfile(source_file_path, target_file_path)

In [ ]:
#split into training data
train = tf.keras.utils.image_dataset_from_directory(
    reduced_data_path,
    validation_split=0.3,  # 70% for training
    subset="training",
    seed=123,
    image_size=(img_height, img_width), #resize to 150x150
    batch_size=batch_size
)


In [ ]:
#split into validation
val = tf.keras.utils.image_dataset_from_directory(
    reduced_data_path,
    validation_split=0.2,  # 20% for validation
    subset="validation",
    seed=123,
    image_size=(img_height, img_width), #resize to 150x150
    batch_size=batch_size
)

In [ ]:
#split into testing
test = tf.keras.utils.image_dataset_from_directory(
    reduced_data_path,
    validation_split=0.1,  # 10% for testing
    subset="validation",  #
    seed=123,
    image_size=(img_height, img_width), #resize to 150x150
    batch_size=batch_size
)

In [ ]:
num_classes = len(class_names)

In [ ]:

# Creating a Sequential model for image classification
model = Sequential([
    # Normalizing pixel values to the range [0,1]
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),

    # Convolutional layers with relu activation and max pooling for feature extraction
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # Flatten layer to prepare for dense layers
    layers.Flatten(),

    # Dense layers with relu activation and dropout for regularization
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    # Output layer with softmax activation for multi-class classification
    layers.Dense(num_classes, activation='softmax')
])


In [ ]:
# Compiling the model with the Adam optimizer, Sparse Categorical Crossentropy loss,
# and accuracy as the evaluation metric
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
#TRAINING
#The following model was first trained with 20 epochs then it colab crashed at the 18th epoch. The model worked with 10 epochs
epochs= 10
history = model.fit(
  train,
  validation_data=val,
  epochs=epochs
)

In [ ]:
# Extracting and plotting accuracy and loss metrics from the training history
acc = history.history['accuracy']
print("accuracy: ", acc)
val_acc = history.history['val_accuracy']
print("val accuracy: ", val_acc)

loss = history.history['loss']
print("loss: ", loss)

val_loss = history.history['val_loss']
print("val loss: ", val_loss)

epochs_range = range(epochs)

# Plotting accuracy
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plotting loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()


In [ ]:
# Evaluate the model on the test set
test_results = model.evaluate(test)

# Print the test results (loss and accuracy)
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])

In [ ]:
# Define Sequential model for the 2nd model with hypertuned parameters
model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),  # Rescale pixel values to [0,1]

    # Convolutional Block 1
    layers.Conv2D(64, 3, padding='same', activation='relu'),  # 64 filters, 3x3 kernel, 'same' padding, ReLU activation
    layers.BatchNormalization(),  # Batch normalization for stabilizing and accelerating training
    layers.MaxPooling2D(),  # Max pooling layer

    # Convolutional Block 2
    layers.Conv2D(128, 3, padding='same', activation='relu'),  # 128 filters, 3x3 kernel, 'same' padding, ReLU activation
    layers.BatchNormalization(),  # Batch normalization for stabilizing and accelerating training
    layers.MaxPooling2D(),  # Max pooling layer

    # Convolutional Block 3
    layers.Conv2D(256, 3, padding='same', activation='relu'),  # 256 filters, 3x3 kernel, 'same' padding, ReLU activation
    layers.BatchNormalization(),  # Batch normalization for stabilizing and accelerating training
    layers.MaxPooling2D(),  # Max pooling layer

    layers.Flatten(),  # Flatten the output for the fully connected layers

    # Fully Connected Layers
    layers.Dense(512, activation='relu'),  # 512 units, ReLU activation
    layers.Dropout(0.5),  # Dropout layer for regularization

    layers.Dense(256, activation='relu'),  # 256 units, ReLU activation
    layers.Dropout(0.5),  # Dropout layer for regularization

    layers.Dense(num_classes, activation='softmax')  # Output layer with softmax activation
])


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#the second model was originally ran with 10 epochs however colab failed after the 5th one. The model works with 6.
epochs= 6
history = model.fit(
  train,
  validation_data=val,
  epochs=epochs
)

In [ ]:
# Extracting and plotting accuracy and loss metrics from the training history
acc = history.history['accuracy']
print("accuracy: ", acc)
val_acc = history.history['val_accuracy']
print("val accuracy: ", val_acc)

loss = history.history['loss']
print("loss: ", loss)

val_loss = history.history['val_loss']
print("val loss: ", val_loss)

epochs_range = range(epochs)

# Plotting accuracy
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plotting loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()


In [ ]:
# Evaluate the model on the test set
test_results = model.evaluate(test)
# Print the test results (loss and accuracy)
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])